In [2]:
import os
import sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(''))))
from config import *

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
from functools import reduce

experiment = "SMOOTH_PURSUITS"

df_event = pd.read_parquet(PREPROCESSED_DIR / f"{experiment}_events.pq")


def rename_columns(df):
    """Renames columns by joining multi-level column names with different delimiters."""
    # Iterate over all column names
    df.columns = [f"{col[0]}" if col[1] == '' else f"{col[0]}_{col[1]}" for col in df.columns.values]
    return df

In [29]:
def get_distance_between_fixations(df: pd.DataFrame) -> pd.DataFrame:
    """Finds acceleration features for anti saccade experiment

    Args:
        df (pd.DataFrame): Dataframe with preprocessed events

    Returns:
        pd.DataFrame: Dataframe with columns ['experiment','participant_id', X_FEATURES]
        where X_FEATURES is a collection of features found by the following cartesian product:
    """

    df = (df.query("event == 'EFIX'")
    .join((df
        .query("event == 'EFIX'")
        .groupby(["experiment", "participant_id", "trial_id", "eye"])[['x','y']].shift(1)
        .rename(columns={"x": "x_lagged", 
            "y": "y_lagged"})))
    .assign(
        x_fixation_dist = lambda x: x["x"] - x["x_lagged"],
        y_fixation_dist = lambda x: x["y"] - x["y_lagged"])
    .assign(
        fixation_distance = lambda x: np.sqrt( np.power(x["x_fixation_dist"],2) + np.power(x["y_fixation_dist"],2))
    )
    .groupby(["experiment", "participant_id"])
    .agg({'fixation_distance': [np.mean, np.std],
    })
    .reset_index()
    .pipe(rename_columns)
    )
    return df
    
    

get_distance_between_fixations(df_event)


,experiment,participant_id,fixation_distance_mean,fixation_distance_std
0,SMOOTH_PURSUITS,106,330.423193,251.631952
1,SMOOTH_PURSUITS,111,258.502899,177.461196
2,SMOOTH_PURSUITS,135,325.349495,156.371913
3,SMOOTH_PURSUITS,142,273.624604,139.110812
4,SMOOTH_PURSUITS,143,280.553196,165.282124
...,...,...,...,...
151,SMOOTH_PURSUITS,399,240.334894,161.850108
152,SMOOTH_PURSUITS,401,332.674424,179.057601
153,SMOOTH_PURSUITS,402,340.382073,203.415434
154,SMOOTH_PURSUITS,403,300.474018,137.125117
